In [1]:
import numpy as np
import pandas as pd

from numba import njit, prange

from utils import *

In [2]:
# Import data
t_costs = pd.read_stata('trade_cost.dta')
t_matrix = pd.read_stata('trade_matrix.dta')
distance = pd.read_stata('exp_distance_gdp.dta')

# Year to int
t_costs['year'] = t_costs['year'].astype(int)

# Part a

After setting $\sigma_\alpha$ and $\sigma_\varepsilon$ to 0, the equation becomes
$$
x_{ji}(\omega_{i,t}) = \frac{(\mu_{ji}\omega_{ji,t})^{-\bar{\epsilon}}}{\sum_{l=1}^N(\mu_{li}\omega_{li,t})^{-\bar{\epsilon}}}
$$
We can transform this to
$$
\ln x_{ji} = -\bar{\epsilon} (\ln \mu_{ji} + \ln\omega_{ji,t}) - \ln \sum_{l=1}^N(\mu_{li}\omega_{li,t})^{-\bar{\epsilon}}
$$
$$
\ln x_{ji} = -\bar{\epsilon} (\ln \mu_{ji} + \ln z_{ji,t} + \phi_{ji} + \varepsilon_{j,t} + \eta_{ji,t}) - \ln \sum_{l=1}^N(\mu_{li}\omega_{li,t})^{-\bar{\epsilon}}
$$
$$
\ln x_{ji} = -\bar{\epsilon} \ln z_{ji,t} + A_{ji} + B_{jt} + C_{it} + \tilde{\eta}_{jit}
$$

In [3]:
#Create the three way dummies
t_costs['A'] = t_costs['exp'].str.cat(t_costs['imp'], sep='_')
t_costs['B'] = t_costs['exp'].str.cat(t_costs['year'].astype(str), sep='_')
t_costs['C'] = t_costs['imp'].str.cat(t_costs['year'].astype(str), sep='_')

# Merge
data_stata = t_matrix.merge(t_costs, on=['year', 'exp', 'imp'], how = 'inner')

#Save it for stata :((
data_stata.to_csv('costs_stata.csv')

```
import delimited costs_stata.csv

gen y = ln(share)

reghdfe y cost, absorb(a b c) vce(cluster imp) nocons
```

Here is the output:

```
. reghdfe y cost, absorb(a b c) vce(cluster imp) nocons
(dropped 32 singleton observations)
(MWFE estimator converged in 2 iterations)
warning: missing F statistic; dropped variables due to collinearity or too few clusters

HDFE Linear regression                            Number of obs   =      1,120
Absorbing 3 HDFE groups                           F(   1,      1) =          .
Statistics robust to heteroskedasticity           Prob > F        =          .
                                                  R-squared       =     0.9941
                                                  Adj R-squared   =     0.9856
                                                  Within R-sq.    =     0.0891
Number of clusters (imp)     =          2         Root MSE        =     0.2403

                                    (Std. Err. adjusted for 2 clusters in imp)
------------------------------------------------------------------------------
             |               Robust
           y |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        cost |  -6.246808   4.81e-15 -1.3e+15   0.000    -6.246808   -6.246808
------------------------------------------------------------------------------

Absorbed degrees of freedom:
-----------------------------------------------------+
 Absorbed FE | Categories  - Redundant  = Num. Coefs |
-------------+---------------------------------------|
           a |        70          70           0    *|
           b |       560           0         560     |
           c |        32          32           0    *|
-----------------------------------------------------+
* = FE nested within cluster; treated as redundant for DoF computation
```

# Part b

## Preparing the data

Here I first construct the data on (log) costs, expenditure shares and (log) GDPs for all countries.

In [4]:
###############################
# Get data to required matrices

# Main data matrix -- start with costs
data = t_costs[['exp', 'imp', 'year', 'cost']].sort_values(['exp', 'imp', 'year'])

# Get missing MFs -- missing costs are only USA-USA and AUS-AUS; this is 0
data = data.set_index(['exp', 'imp', 'year'])
new_ind = pd.MultiIndex.from_product(data.index.levels, names=['exp', 'imp', 'year'])
data = data.reindex(new_ind, fill_value=0).reset_index()

# Merge with shares
data = data.merge(t_matrix, on = ['exp', 'imp', 'year']).drop(columns = ['value', 'tot_production', 'tot_expenditure'])

# Merge with distance
dists = pd.melt(distance, ['exp', 'lgdp'], var_name = 'imp', value_name='dist')
dists['imp'] = dists['imp'].str.extract(r'.*_(\w{3})')
data = data.merge(dists[['exp', 'imp', 'dist']], on = ['exp', 'imp'])

# GDP
GDP = distance[['exp', 'lgdp']].sort_values('exp')

# Number of exporters, importers, and time periods
E, I, T = data[['exp', 'imp', 'year']].nunique()

Here I just transform the previous (pandas) data to numpy, making `X` (exp. shares) into a 3D matrix ($E\times I \times T$) to feed into my optimization function. I also create the $E \times 1$ column of log GDPs `K`.

In [5]:
############################################################################
# Ok, now that we have the data together and sorted, let's get it to numpyyy

# Start with shares
X = data['share'].values.reshape((E, I, T), order='C')

# Consistency check
exp_10, imp_1, year_5 = GDP['exp'].iloc[10], data['imp'].unique()[1], data['year'].iloc[5]
assert(data.query('exp==@exp_10 & imp==@imp_1 & year == @year_5')['share'].iat[0] == X[10, 1, 5])

# GDP
K = GDP['lgdp'].values

Here I constuct the matrix `z` of $\tilde{z}_{ij}$ values. As it says in the problem set, this is a $ET \times 1$ vector, where $E$ is the number of exporters, and so `E` is the major row index in `z`.

When constructing `z`, I purge all entries with USA as an importer, because we will not need them in the further analysis (all such $\tilde{z}_{ij}$ would be 0).

In [6]:
# Get the costs -- don't purge USA just yet
costs = data[['exp', 'imp', 'year', 'cost']].copy()

# Get the relative Zs
costs['cost'] = costs.groupby(['year', 'imp'])['cost'].transform(lambda x: x - x.iloc[-1]) -\
                    costs.groupby(['year', 'exp'])['cost'].transform(lambda x: x.iloc[-1])

# Purge USA as imp
costs = costs.query('imp != "USA"').drop(columns = ['imp'])

# Consistency check
c_IDN_2000 = data.query('exp=="IDN" & imp=="AUS" & year==2000').iat[0, 3]
c_IDN_2000 += -data.query('exp=="USA" & imp=="AUS" & year==2000').iat[0, 3]
c_IDN_2000 += -data.query('exp=="IDN" & imp=="USA" & year==2000').iat[0, 3]

assert(costs.query('exp=="IDN" & year==2000').iat[0, 2] == c_IDN_2000)    

# Alright, to numpy
z = costs['cost'].values.reshape((costs.shape[0], 1))

Here I construct the `Z` and `Y` matrices (`Y` corresponds to `X` in the problem set), where `Z` is just `z` + a matrix of dummy variables $d_{ji}$ (essentialy $d_j$), and `Y` is `Z` plus a $ET \times E$ matrix of $\{|\ln \kappa_j - \ln \kappa_l|(\ln z_{li,t} - \ln z_{l1,t})\}$.

In [7]:
# Get dummies and construct Z
dummies = pd.get_dummies(costs['exp']).values
Y = np.concatenate((z, dummies), axis=1)

# Start construction of abs_K -- matrix for |k_j - k_l|
K_l = K.reshape((1, K.size)).repeat(Y.shape[0], axis = 0)
K_j = K.repeat(T).reshape((E*T, 1))
abs_K = np.abs(K_j - K_l)

# Now construct (ln z_{li,t} - ln z_{l1,t})
costs_li = data[['exp', 'imp', 'year', 'cost']].query('imp != "USA"').drop(columns = ['imp'])
z_li = costs_li.sort_values(['year', 'exp'])['cost'].values.reshape((T,E))
z_li = z_li.repeat(E, axis=0)

costs_l1 = data[['exp', 'imp', 'year', 'cost']].query('imp == "USA"').drop(columns = ['imp'])
z_l1 = costs_l1.sort_values(['year', 'exp'])['cost'].values.reshape((T,E))
z_l1 = z_l1.repeat(E, axis=0)

Z_l = z_li - z_l1

# Construct the last part of Z matrix and concatenate to get Y
Z = np.concatenate((Y, abs_K*Z_l), axis = 1)

## Computations

First, I set up initial parameters $\sigma_\alpha$ and $\sigma_\epsilon$, and create the matrix of simulated random values for $(\alpha_S, \ln \epsilon_S)$. I initialize the initial guess for $\delta_{ji,t}$ matrix to 0.

In [8]:
# Random matrixx
S = np.random.randn(200, 2)
S[:, 1] = np.exp(S[:, 1])

# Initial guess matrix D (will be changed later)
D_0 = np.zeros_like(X)

def get_loss(args):    
    sigma_a, sigma_e = args.flatten()
    
    # I compute the delta_{ji,t} matrix
    _, D_0 = shares_converge(X, K, np.zeros_like(X), S, sigma_a, sigma_e)
    
    # First, I compute delta_{ji,t} - delta_{j1,t}  and remove all delta_{ji,t} 
    # entries where i==USA (last index of i), since these things would now fall out,
    # then reshape to 1 column vector
    D = D_0[:, 0, :] - D_0[:, -1, :] 
    D = D.reshape((D.size, 1))
    
    # Compute theta_1
    inv = np.linalg.inv(Z.T @ Z)
    M1 = Y.T @ Z @ inv @ Z.T
    T_1 = np.linalg.inv( M1 @ Y) @ M1 @ D

    # Yep, the pset fucked this up
    T_1[0][0] = -T_1[0][0]
    
    # This function computes the e() function, given the matrices
    def error(e_mu, z, D):
        return D + e_mu[0,0] * z - e_mu[1:,:].repeat(T, axis = 0)

    def loss(e_mu, z, Z, D):
        e_mu = e_mu.reshape((e_mu.size, 1))
        return error(e_mu, z, D).T @ Z @ np.linalg.inv(Z.T @ Z) @ Z.T @ error(e_mu, z, D)
    
    l = loss(T_1, z, Z, D).flatten()
    print(l, sigma_a, sigma_e)
    
    return loss(T_1, z, Z, D)

In [ ]:
from scipy.optimize import minimize

# Set up parameters
sigma_a, sigma_e = 1.1, 1.1

minimize(get_loss, np.array([sigma_a, sigma_e]), method = 'Nelder-Mead')

In [11]:
get_loss(np.array([2, 0.003]))

[nan] 2.0 0.003


array([[nan]])